In [17]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, AlphaDropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [18]:
import os
import pickle
import numpy as np

In [19]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [20]:
# list devices so you can check whether your gpu is available
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [21]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [22]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:],kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('selu'))
model.add(Conv2D(32, (3, 3),kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('selu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(AlphaDropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same',kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('selu'))
model.add(Conv2D(64, (3, 3),kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('selu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(AlphaDropout(0.1))

model.add(Flatten())
model.add(Dense(512,kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('selu'))
model.add(AlphaDropout(0.2))
model.add(Dense(num_classes,kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('softmax'))

In [23]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Using real-time data augmentation.
Epoch 1/10
1562/1562 [==============================] - 117s 74ms/step - loss: 1.9012 - accuracy: 0.3162 - val_loss: 1.9002 - val_accuracy: 0.4259
Epoch 2/10
1562/1562 [==============================] - 118s 76ms/step - loss: 1.6100 - accuracy: 0.4191 - val_loss: 1.7416 - val_accuracy: 0.4879
Epoch 3/10
1562/1562 [==============================] - 118s 75ms/step - loss: 1.4755 - accuracy: 0.4699 - val_loss: 1.6425 - val_accuracy: 0.5466
Epoch 4/10
1562/1562 [==============================] - 117s 75ms/step - loss: 1.3773 - accuracy: 0.5070 - val_loss: 1.3664 - val_accuracy: 0.5702
Epoch 5/10
1562/1562 [==============================] - 117s 75ms/step - loss: 1.3050 - accuracy: 0.5348 - val_loss: 1.3543 - val_accuracy: 0.6000
Epoch 6/10
1562/1562 [==============================] - 117s 75ms/step - loss: 1.2441 - accuracy: 0.5592 - val_loss: 1.2486 - val_accuracy: 0.6190
Epoch 7/10
1562/1562 [==============================] - 117s 75ms/step - loss: 1.18

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [24]:
# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

print('Model Accuracy = %.5f' % (evaluation[1]))


312/312 [==============================] - 7s 20ms/step - loss: 1.1890 - accuracy: 0.6505
Model Accuracy = 0.65054


In [25]:
f = open('CIFAR10_SELU_results.txt', 'a')
f.write(' Test accuracy:' + str(evaluation[1]) +  '\n')  
f.close() 

predict_gen = model.predict(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

312/312 [==============================] - 7s 21ms/step
Actual Label = cat vs. Predicted Label = truck
Actual Label = ship vs. Predicted Label = automobile
Actual Label = ship vs. Predicted Label = ship
Actual Label = airplane vs. Predicted Label = cat
Actual Label = frog vs. Predicted Label = automobile
Actual Label = frog vs. Predicted Label = airplane
Actual Label = automobile vs. Predicted Label = airplane
Actual Label = frog vs. Predicted Label = airplane
Actual Label = cat vs. Predicted Label = automobile
Actual Label = automobile vs. Predicted Label = ship
Actual Label = airplane vs. Predicted Label = ship
Actual Label = truck vs. Predicted Label = truck
Actual Label = dog vs. Predicted Label = cat
Actual Label = horse vs. Predicted Label = bird
Actual Label = truck vs. Predicted Label = ship
Actual Label = ship vs. Predicted Label = dog
Actual Label = dog vs. Predicted Label = cat
Actual Label = horse vs. Predicted Label = ship
Actual Label = ship vs. Predicted Label = airplane

In [28]:
# Load the saved model with 300 iterations and evaluation
import tensorflow  as tf
model = tf.keras.models.load_model('./saved_models/keras_cifar10_trained_model_300it.h5')

evaluation = model.evaluate(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

print('Model Accuracy = %.5f' % (evaluation[1]))


312/312 [==============================] - 6s 19ms/step - loss: 1.0798 - accuracy: 0.8151
Model Accuracy = 0.81510
